In [11]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import pywt

In [12]:
def load_images(folder):
    imgs = []
    for f in sorted(os.listdir(folder)):
        if f.lower().endswith((".jpg",".png",".jpeg")):
            img = cv2.imread(os.path.join(folder,f), cv2.IMREAD_GRAYSCALE)
            imgs.append(img.astype(np.float32))
    return imgs

In [13]:
def center_crop(img, size=2048):
    h, w = img.shape
    size = min(size, h, w)
    half = size // 2
    return img[h//2-half:h//2+half, w//2-half:w//2+half]

def extract_residual_wavelet(img):
    img = center_crop(img)

    # Wavelet decomposition
    coeffs = pywt.wavedec2(img, wavelet='db8', level=1)

    # Remove approximation (scene)
    coeffs[0] = np.zeros_like(coeffs[0])

    # Reconstruct noise
    r = pywt.waverec2(coeffs, wavelet='db8')

    return r - np.mean(r)

In [14]:
def compute_prnu(images):
    acc = None
    for img in tqdm(images):
        r = extract_residual_wavelet(img)
        acc = r if acc is None else acc + r
    return acc / len(images)

In [15]:
cam1 = load_images(r"C:\Users\Saif Kazi\OneDrive\Desktop\dataset\real\camera_1")
cam2 = load_images(r"C:\Users\Saif Kazi\OneDrive\Desktop\dataset\real\camera_2")

In [16]:
prnu1 = compute_prnu(cam1)
prnu2 = compute_prnu(cam2)

  0%|          | 0/67 [00:00<?, ?it/s]

100%|██████████| 73/73 [00:29<00:00,  2.46it/s]


In [17]:
def correlate(img, prnu):
    r = extract_residual_wavelet(img)
    p = prnu - np.mean(prnu)
    return np.sum(r*p) / (np.linalg.norm(r)*np.linalg.norm(p) + 1e-8)

In [18]:
corr_self = (
    [correlate(img, prnu1) for img in cam1] +
    [correlate(img, prnu2) for img in cam2]
)

corr_cross = (
    [correlate(img, prnu2) for img in cam1] +
    [correlate(img, prnu1) for img in cam2]
)

T_low  = np.percentile(corr_cross, 99)
T_high = np.percentile(corr_self, 5)

print("T_low :", T_low)
print("T_high:", T_high)

T_low : 0.0033671889
T_high: 0.03222487


In [19]:
os.makedirs("prnu", exist_ok=True)

np.save("prnu/prnu_cam1.npy", prnu1)
np.save("prnu/prnu_cam2.npy", prnu2)
np.save("prnu/T_low.npy", T_low)
np.save("prnu/T_high.npy", T_high)

print("PRNU model saved")

PRNU model saved
